Hadoop/MapReduce -- WordCount en Python (Hadoop Streaming)
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/big-data-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/big-data-analytics/tree/master/) para explorar el repositorio usando `nbviewer`.

---

# Definición del problema

Se desea contar la frecuencia de ocurrencia de palabras en un conjunto de documentos. Debido a los requerimientos de diseño (gran volúmen de datos y tiempos rápidos de respuesta) se desea implementar una arquitectura Big Data. Se desea implementar la solución en **Python** .

A continuación se generarán tres archivos de prueba para probar el sistema.

In [1]:
## Se crea el directorio de entrada
!rm -rf input
!rm -rf output
!mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


# Solución

### Prueba de la implementación

Para realizar la implementación solicitada se usará Hadoop Streaming, el cual permite la implementación de aplicaciones MapReduce en cualquier lenguaje que soporte la creación de scripts. Para este caso se ejemplificará la implementación en Python. Hadoop streaming debe su nombre a que usa los streamings de Unix para la entrada y salida en texto (stdin, stdout y stderr respectivamente). El sistema lee y escribe líneas de texto una a una a los respectivos flujos de entrada y salida, tal como ocurre de forma estándar en las herramientas de la línea de comandos de Linux. 

La implementación requiere un programa para realizar el mapeo y otro para la reducción que se ejecutan independientemente; Hadoop se encarga de la coordinación entre ellos. El intercambio de información se hace a traves de texto, que es el lenguaje universal para intercambio de información. 

In [5]:
%%writefile mapper.py
#! /usr/bin/env python

##
## Esta es la función que mapea la entrada a parejas (clave, valor)
##
import sys
if __name__ == "__main__": 
    
    ##
    ## itera sobre cada linea de código recibida
    ## a través del flujo de entrada
    ##
    for line in sys.stdin:
        
        ##
        ## genera las tuplas palabra \tabulador 1
        ## ya que es un conteo de palabras
        ##
        for word in line.split(): 
                   
            ##
            ## escribe al flujo estándar de salida
            ##
            sys.stdout.write("{}\t1\n".format(word))
            

Writing mapper.py


In [6]:
## El programa anterior se hace ejecutable
!chmod +x mapper.py 

In [7]:
## la salida de la función anterior es:
!cat ./input/text*.txt | ./mapper.py | head

Analytics	1
is	1
the	1
discovery,	1
interpretation,	1
and	1
communication	1
of	1
meaningful	1
patterns	1


In [8]:
%%writefile reducer.py
#!/usr/bin/env python

import sys

##
## Esta función reduce los elementos que 
## tienen la misma clave
##

if __name__ == '__main__': 
  
    curkey = None
    total = 0
    
    ##
    ## cada linea de texto recibida es una 
    ## entrada clave \tabulador valor
    ##
    for line in sys.stdin:
        
        key, val = line.split("\t") 
        val = int(val)
        
        if key == curkey: 
            ##
            ## No se ha cambiado de clave. Aca se 
            ## acumulan los valores para la misma clave.
            ##
            total += val  
        else:
            ##
            ## Se cambio de clave. Se reinicia el
            ## acumulador.
            ##
            if curkey is not None:
                ##
                ## una vez se han reducido todos los elementos
                ## con la misma clave se imprime el resultado en
                ## el flujo de salida
                ##
                sys.stdout.write("{}\t{}\n".format(curkey, total)) 
            
            curkey = key
            total = val

Writing reducer.py


In [9]:
!chmod +x reducer.py

In [10]:
##
## La función sort hace que todos los elementos con 
## la misma clave queden en lineas consecutivas.
## Hace el papel del módulo Shuffle & Sort
##
!cat ./input/text*.txt | ./mapper.py | sort | ./reducer.py | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	2
Especially	1
Organizations	1
Since	1
Specifically,	1


### Ejecución en Hadoop

Una vez se tienen las versiones anteriores funcionando, se puede proceder a ejecutar la tarea directamente en hadoop. 

In [11]:
## Primero se limpian los directorios.
!rm -rf output

In [12]:
##
## Se ejecuta en Hadoop.
##   -input: archivo de entrada
##   -output: directorio de salida
##   -maper: programa que ejecuta el map
##   -reducer: programa que ejecuta la reducción
##
!hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar -input input -output output  -mapper mapper.py -reducer reducer.py

2018-08-27 20:34:34,881 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-08-27 20:34:35,283 INFO impl.MetricsConfig: loaded properties from hadoop-metrics2.properties
2018-08-27 20:34:35,341 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2018-08-27 20:34:35,342 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2018-08-27 20:34:35,358 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2018-08-27 20:34:35,639 INFO mapred.FileInputFormat: Total input files to process : 3
2018-08-27 20:34:35,689 INFO mapreduce.JobSubmitter: number of splits:3
2018-08-27 20:34:35,838 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local301458228_0001
2018-08-27 20:34:35,839 INFO mapreduce.JobSubmitter: Executing with tokens: []
2018-08-27 20:34:35,939 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2018-08-27 20:34:35,940

2018-08-27 20:34:36,368 INFO mapred.MapTask: (EQUATOR) 0 kvi 26214396(104857584)
2018-08-27 20:34:36,368 INFO mapred.MapTask: mapreduce.task.io.sort.mb: 100
2018-08-27 20:34:36,368 INFO mapred.MapTask: soft limit at 83886080
2018-08-27 20:34:36,368 INFO mapred.MapTask: bufstart = 0; bufvoid = 104857600
2018-08-27 20:34:36,368 INFO mapred.MapTask: kvstart = 26214396; length = 6553600
2018-08-27 20:34:36,369 INFO mapred.MapTask: Map output collector class = org.apache.hadoop.mapred.MapTask$MapOutputBuffer
2018-08-27 20:34:36,374 INFO streaming.PipeMapRed: PipeMapRed exec [/Volumes/JetDrive/GitHub/big-data-analytics/./mapper.py]
2018-08-27 20:34:36,381 INFO streaming.PipeMapRed: R/W/S=1/0/0 in:NA [rec/s] out:NA [rec/s]
2018-08-27 20:34:36,421 INFO streaming.PipeMapRed: Records R/W=4/1
2018-08-27 20:34:36,424 INFO streaming.PipeMapRed: MRErrorThread done
2018-08-27 20:34:36,425 INFO streaming.PipeMapRed: mapRedFinished
2018-08-27 20:34:36,425 INFO mapred.LocalJobRunner: 
2018-08-27 20:34:3

2018-08-27 20:34:36,949 INFO mapreduce.Job: Job job_local301458228_0001 running in uber mode : false
2018-08-27 20:34:36,950 INFO mapreduce.Job:  map 100% reduce 100%
2018-08-27 20:34:36,951 INFO mapreduce.Job: Job job_local301458228_0001 completed successfully
2018-08-27 20:34:36,959 INFO mapreduce.Job: Counters: 30
	File System Counters
		FILE: Number of bytes read=721072
		FILE: Number of bytes written=2728932
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=24
		Map output records=252
		Map output bytes=2369
		Map output materialized bytes=2891
		Input split bytes=348
		Combine input records=0
		Combine output records=0
		Reduce input groups=159
		Reduce shuffle bytes=2891
		Reduce input records=252
		Reduce output records=158
		Spilled Records=504
		Shuffled Maps =3
		Failed Shuffles=0
		Merged Map outputs=3
		GC time elapsed (ms)=26
		Total committed heap usage (bytes)=1296

In [13]:
## contenido del directorio con los 
## resultados de la corrida
!ls output/

_SUCCESS   part-00000


In [14]:
## se visualiza el archivo con los
## resultados de la corrida
!cat output/part-00000 | head

(DA)	1
(see	1
Analytics	2
Analytics,	1
Big	1
Data	3
Especially	1
Organizations	1
Since	1
Specifically,	1


In [15]:
!rm reducer.py mapper.py
!rm -rf input output

# Notas.

**Combiners.--** Los combiners son *reducers* que se ejecutan sobre los resultdos que produce cada mapper antes de pasar al modulo de suffle-&-sort, con el fin de reducir la carga computacional. Suelen ser identicos a los *reducers*.

**Partitioners.--** Son rutinas que controlan como se enviar las parejas (clave, valor) a cada reducers, tal que elementos con la misma clave son enviados al mismo reducer. 

**Job Chain.--** Se refiere al encadenamiento de varias tareas cuando el cómputo que se desea realizar es muy complejo para que pueda realizarse en un MapReduce.

---

Hadoop/MapReduce -- WordCount en Python (Hadoop Streaming)
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/big-data-analytics/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/big-data-analytics/tree/master/) para explorar el repositorio usando `nbviewer`.